<a href="https://colab.research.google.com/github/sayakpaul/CI-CD-for-Model-Training/blob/main/cloud_function_trigger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## References

* https://github.com/GoogleCloudPlatform/mlops-with-vertex-ai/blob/main/05-continuous-training.ipynb

## Setting up

In [ ]:
!pip install --upgrade -q google-cloud-scheduler

### ***Restart runtime.***

In [ ]:
!gcloud init

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [198]:
GOOGLE_CLOUD_PROJECT = "gcp-ml-172005"
GOOGLE_CLOUD_REGION = "us-central1"

PUBSUB_TOPIC = f"trigger-{PIPELINE_NAME}"
SCHEDULER_JOB_NAME = "MLOpsJob"

In [95]:
!export GOOGLE_APPLICATION_CREDENTIALS="/home/jupyter/CI-CD-for-Model-Training/gcp-ml-172005-528977a75f85.json"

## Create Pub/Sub Topic via CLI

In [172]:
import json

data = '{"num_epochs": "3", "learning_rate": "1e-2"}'
data = json.dumps(data)

In [173]:
!gcloud scheduler jobs create pubsub $SCHEDULER_JOB_NAME --schedule "*/3 * * * *" --topic $PUBSUB_TOPIC --message-body $data

name: projects/gcp-ml-172005/locations/us-central1/jobs/MLOpsJob
pubsubTarget:
  data: eyJudW1fZXBvY2hzIjogIjMiLCAibGVhcm5pbmdfcmF0ZSI6ICIxZS0yIn0=
  topicName: projects/gcp-ml-172005/topics/trigger-penguin-vertex-training
retryConfig:
  maxBackoffDuration: 3600s
  maxDoublings: 16
  maxRetryDuration: 0s
  minBackoffDuration: 5s
schedule: '*/3 * * * *'
state: ENABLED
timeZone: Etc/UTC
userUpdateTime: '2021-08-26T17:41:58Z'


### See Its behaviour in Vertex AI Pipeline

![](https://i.ibb.co/GkHmwTL/Screen-Shot-2021-08-27-at-2-49-00-AM.png)

## Create Pub/Sub Topic Programatically

In [199]:
import json
from google.oauth2 import service_account
from google.cloud.scheduler_v1.services import cloud_scheduler
from google.cloud.scheduler_v1.types.job import Job
from google.cloud.scheduler_v1.types import PubsubTarget
from google.cloud.scheduler_v1.types.cloudscheduler import CreateJobRequest

credentials = service_account.Credentials.from_service_account_file("/home/jupyter/CI-CD-for-Model-Training/gcp-ml-172005-528977a75f85.json")
client = cloud_scheduler.CloudSchedulerClient(credentials=credentials)

In [217]:
data = {"num_epochs": "3", "learning_rate": "1e-2"}
data = json.dumps(data).encode('utf-8')

pubsub_target = PubsubTarget(topic_name=PUBSUB_TOPIC, data=data)

job = Job(name=SCHEDULER_JOB_NAME, 
          pubsub_target=pubsub_target, 
          schedule="0 */3 * * *")

req = CreateJobRequest(parent=???, job=job)

In [219]:
result_job = client.create_job(req)

PermissionDenied: 403 The principal (user or service account) lacks IAM permission "cloudscheduler.jobs.create" for the resource "gcp-ml-172005" (or the resource may not exist).